In [1]:
import pandas as pd
import numpy as np
import openpyxl
from sklearn import ensemble
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
import pickle, os
import warnings
warnings.filterwarnings('ignore')

# Reading Combined Dataset

In [2]:
df = pd.read_excel('./datasets/combined.xlsx', index_col=0)
df.head()

,Dwelling Type,Year,Month,Region,Towns,Avg kWh,Daily Rainfall Total (mm),Highest 30 min Rainfall (mm),Highest 60 min Rainfall (mm),Highest 120 min Rainfall (mm),Mean Temperature (°C),Maximum Temperature (°C),Minimum Temperature (°C),Mean Wind Speed (km/h),Max Wind Speed (km/h)
0,1-room / 2-room,2005,1,Central Region,Bishan,104.9,2.9,3.6,4.4,5.0,27.5,31.3,25.1,7.2,35.3
1,1-room / 2-room,2005,1,Central Region,Bukit Merah,140.7,2.9,3.6,4.4,5.0,27.5,31.3,25.1,7.2,35.3
2,1-room / 2-room,2005,1,Central Region,Central Region,136.5,2.9,3.6,4.4,5.0,27.5,31.3,25.1,7.2,35.3
3,1-room / 2-room,2005,1,Central Region,Geylang,148.5,2.9,3.6,4.4,5.0,27.5,31.3,25.1,7.2,35.3
4,1-room / 2-room,2005,1,Central Region,Kallang,115.6,2.9,3.6,4.4,5.0,27.5,31.3,25.1,7.2,35.3


# One Hot Encoding Of Dataset

In [3]:
df_one_hot_encoded = df.copy()

In [4]:
features_df = pd.get_dummies(df_one_hot_encoded, columns=['Dwelling Type', 'Month', 'Towns', 'Region'])
del features_df['Avg kWh']
del features_df['Highest 30 min Rainfall (mm)']
del features_df['Highest 60 min Rainfall (mm)']
features_df

,Year,Daily Rainfall Total (mm),Highest 120 min Rainfall (mm),Mean Temperature (°C),Maximum Temperature (°C),Minimum Temperature (°C),Mean Wind Speed (km/h),Max Wind Speed (km/h),Dwelling Type_1-room / 2-room,Dwelling Type_3-room,...,Towns_Tanglin,Towns_Toa Payoh,Towns_West Region,Towns_Woodlands,Towns_Yishun,Region_Central Region,Region_East Region,Region_North East Region,Region_North Region,Region_West Region
0,2005,2.9,5.0,27.5,31.3,25.1,7.2,35.3,1,0,...,0,0,0,0,0,1,0,0,0,0
1,2005,2.9,5.0,27.5,31.3,25.1,7.2,35.3,1,0,...,0,0,0,0,0,1,0,0,0,0
2,2005,2.9,5.0,27.5,31.3,25.1,7.2,35.3,1,0,...,0,0,0,0,0,1,0,0,0,0
3,2005,2.9,5.0,27.5,31.3,25.1,7.2,35.3,1,0,...,0,0,0,0,0,1,0,0,0,0
4,2005,2.9,5.0,27.5,31.3,25.1,7.2,35.3,1,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60563,2021,9.8,9.4,28.1,32.7,24.5,8.6,31.9,0,0,...,0,0,0,0,0,0,0,0,0,1
60564,2021,7.5,6.7,28.2,32.1,25.2,4.9,26.5,0,0,...,0,0,0,0,0,0,0,0,0,1
60565,2021,8.5,7.9,28.2,32.4,24.9,6.7,29.2,0,0,...,0,0,0,0,0,0,0,0,0,1
60566,2021,8.5,7.9,28.2,32.4,24.9,6.7,29.2,0,0,...,0,0,0,0,0,0,0,0,0,1


# Splitting Of Train Test Datasets

In [5]:
# apply normalization techniques
for column in features_df.columns:
    features_df[column] = (features_df[column] - features_df[column].min()) / (features_df[column].max() - features_df[column].min())

In [6]:
# Training and testing only accept matrix not data frame
X = features_df
y = df_one_hot_encoded['Avg kWh']

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [23]:
eval_results = {}
def perform_eval(model, model_name):
    #Mean squared error 
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    #Train
    mse_train = mean_squared_error(y_train, y_train_pred)
    rmse_train = mean_squared_error(y_train, y_train_pred, squared=False)
    #Test
    mse_test = mean_squared_error(y_test, y_test_pred)
    rmse_test = mean_squared_error(y_test, y_test_pred, squared=False)

    #R Squared Score
    #Train
    r2train = model.score(X_train, y_train)
    adjr2_train = 1 - (1-r2train) * (len(y)-1)/(len(y)-X.shape[1]-1)
    #Test
    r2test = model.score(X_test, y_test)
    adjr2_test = 1 - (1-r2test) * (len(y)-1)/(len(y)-X.shape[1]-1)
    
    eval_results[model_name] = {
        "MSE (Train)" : mse_train,
        "MSE (Test)" : mse_test,
        "RMSE (Train)" : rmse_train,
        "RMSE (Test)" : rmse_test,
        "R2 (Train)" : r2train,
        "R2 (Test)" : r2test,
        "Adj R2 (Train)" : adjr2_train,
        "Adj R2 (Test)" : adjr2_test
    }
    
    return eval_results[model_name]

In [14]:
best_model = ""
def get_best_model(model, best_model):
    if best_model == "":
        best_model = model

    #test adj r2
    best_adjr2_test = 1 - (1-best_model.score(X_test, y_test)) * (len(y)-1)/(len(y)-X.shape[1]-1)
    model_adjr2_test = 1 - (1-model.score(X_test, y_test)) * (len(y)-1)/(len(y)-X.shape[1]-1)

    if model_adjr2_test > best_adjr2_test:
        best_model = model

    return best_model

In [20]:
def compare_results(desired_model):
    metric_data = []
    col_names = ["Models"]
    col_done = False

    for model in eval_results:
        eval_list = []
        #Append Model names
        if (desired_model.lower() in str(model).lower()) or (desired_model.lower() == "all"):
            eval_list.append(model)
            for metric in eval_results[model]:
                if not col_done:
                    col_names.append(metric)
                eval_list.append(eval_results[model][metric])
            col_done = True
            metric_data.append(eval_list)
        
    df = pd.DataFrame(metric_data, columns=col_names)
    df = df.sort_values(by=['Adj R2 (Test)'], ascending=False)
    df = df.style.set_table_attributes("style='display:inline'").set_caption(f'{str(desired_model).capitalize()} Models (Sort by Adj R2 (Test))')
    
    return df

# Initial Ensemble Gradient Boosting Regressor

In [11]:
from sklearn import ensemble
# Fit regression model
base_gbr = ensemble.GradientBoostingRegressor(random_state=7) # default
base_gbr.fit(X_train, y_train)

GradientBoostingRegressor(random_state=7)

In [24]:
best_model = get_best_model(base_gbr, best_model)
res_val = perform_eval(base_gbr, "GBR (Initial)")
for key in res_val:
    print(f"{key}: {res_val[key]}")

MSE (Train): 16402.69977305449
MSE (Test): 15555.0390406328
RMSE (Train): 128.07302515773762
RMSE (Test): 124.71984220897973
R2 (Train): 0.9116433160616547
R2 (Test): 0.9130210625856862
Adj R2 (Train): 0.9115264556666101
Adj R2 (Test): 0.9129060243957752


# Gridsearch to find the best params

In [42]:
from sklearn import ensemble
from sklearn.model_selection import GridSearchCV

# Parameters we want to try
param_grid = {
    'n_estimators': [500, 1000, 3000],
    'max_depth': [4, 6, 9],
    'min_samples_leaf': [3, 5, 9, 15],
    'learning_rate': [1, 0.5, 0.1],
    'max_features': [1.0, 0.3, 0.1],
    'loss': ['squared_error', 'absolute_error', 'huber']
}
# Create the model
model = ensemble.GradientBoostingRegressor()

# Define the grid search we want to run. Run it with four cpus in parallel.
gs_cv = GridSearchCV(model, param_grid, n_jobs=8, verbose=100)

# Run the grid search - on only the training data!
gs_cv.fit(X_train, y_train)

Fitting 5 folds for each of 972 candidates, totalling 4860 fits


GridSearchCV(estimator=GradientBoostingRegressor(), n_jobs=8,
             param_grid={'learning_rate': [1, 0.5, 0.1],
                         'loss': ['squared_error', 'absolute_error', 'huber'],
                         'max_depth': [4, 6, 9],
                         'max_features': [1.0, 0.3, 0.1],
                         'min_samples_leaf': [3, 5, 9, 15],
                         'n_estimators': [500, 1000, 3000]},
             verbose=100)

![alt text](gridsearch.png "GridSearchCV Results")

# It took use around 4905 minutes to run our gridsearchcv which is about 82 hours or over 3 days for running non-stop

In [43]:
# Print the parameters that gave us the best result!
# second run results. (slightly different results)
print(gs_cv.best_params_)

{'learning_rate': 0.1, 'loss': 'huber', 'max_depth': 9, 'max_features': 1.0, 'min_samples_leaf': 15, 'n_estimators': 3000}


In [17]:
from sklearn import ensemble
# Fit regression model
ensemble_gbr = ensemble.GradientBoostingRegressor(
    n_estimators=3000, #how many decision trees to build
    learning_rate=0.1, #how much decision trees influence overall prediction
    max_depth=9,
    min_samples_leaf=15,
    max_features=1.0,
    loss='huber',
    random_state=7
)
ensemble_gbr.fit(X_train, y_train)

GradientBoostingRegressor(loss='huber', max_depth=9, max_features=1.0,
                          min_samples_leaf=15, n_estimators=3000,
                          random_state=7)

In [25]:
best_model = get_best_model(ensemble_gbr, best_model)
res_val = perform_eval(ensemble_gbr, "GBR (Best Params)")
for key in res_val:
    print(f"{key}: {res_val[key]}")

MSE (Train): 2170.229801254006
MSE (Test): 3204.7523371997863
RMSE (Train): 46.58572529492275
RMSE (Test): 56.61053203424065
R2 (Train): 0.9883095885874847
R2 (Test): 0.9820800222848983
Adj R2 (Train): 0.9882941268698759
Adj R2 (Test): 0.9820563213538354


In [26]:
gbr_df = compare_results("gbr") 
gbr_df

,Models,MSE (Train),MSE (Test),RMSE (Train),RMSE (Test),R2 (Train),R2 (Test),Adj R2 (Train),Adj R2 (Test)
1,GBR (Best Params),2170.229801,3204.752337,46.585725,56.610532,0.988310,0.982080,0.988294,0.982056
0,GBR (Initial),16402.699773,15555.039041,128.073025,124.719842,0.911643,0.913021,0.911526,0.912906


<b> Here, we save our trained model first for future use </b>

In [14]:
import pickle, os
filename = 'ensemble_gbr.pkl'
save_location = os.path.join(".","trained_models", filename)
pickle.dump(ensemble_gbr, open(save_location, 'wb'))